# Import Statements

In [ ]:
# !pip freeze >> requirements.txt
!pip uninstall opencv-python==3.4.2.17 
!pip install opencv-python==3.4.2.17 
!pip install opencv-contrib-python==3.4.2.17 

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)? 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import joblib
import pickle
import pandas as pd 
from google.colab.patches import cv2_imshow
import shutil
import scipy.cluster.vq as vq
from scipy.spatial import distance
from scipy import spatial
import random

In [ ]:
import cv2
from sklearn.cluster import KMeans
sift = cv2.xfeatures2d.SIFT_create()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
save_path = '/content/drive/MyDrive/SUB/PROJECTS/IR/Ml_files/instre/'

# Saving and Loading models


In [ ]:
# Saving and Loading models using joblib
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)


# Data Paths 

In [ ]:
dataset_root = '/content/drive/MyDrive/SUB/PROJECTS/IR/INSTRE/'
original_train_folder_name = 'raw_data/data/'
original_test_folder_name = 'raw_data/query_images/'
converted_train_128_folder_name = 'raw_data/Instre_128/'
converted_test_128_folder_name = 'raw_data/query_128/'

In [ ]:
x_dim, y_dim = 128, 128

## Resizing Images and saving names and representations 

In [ ]:
# def read_images(root_folder, source_folder, destination_folder):
#   format = [".jpg",".png",".jpeg"]
#   images = []
#   path = os.path.join(root_folder, source_folder)
#   for img_name in os.listdir(path):
#     if img_name.endswith(tuple(format)):
#       image = cv2.imread(os.path.join(path,img_name))
#       converted_image = cv2.resize(image, (x_dim, y_dim))
#       images.append(converted_image)
#       os.chdir(root_folder+destination_folder)
#       cv2.imwrite(img_name, converted_image)
         
#   return np.array(images)
# train_128 = read_images(dataset_root, original_train_folder_name, converted_train_128_folder_name)
# test_128 = read_images(dataset_root, original_test_folder_name, converted_test_128_folder_name)

In [ ]:
# for path in os.listdir(dataset_root):
#   if path.endswith('.jpg'):
#     shutil.move(dataset_root+path, dataset_original_train + path) 

In [ ]:
def load_images_from_folder(folder):
    images = []
    names = []
    for filename in os.listdir(folder):
      path = folder + "/" + filename
      img = cv2.imread(path, 0)
      if img is not None:
        images.append(img)
        names.append(filename)
    return images, names

In [ ]:
train_128, train_names = load_images_from_folder(dataset_root +  converted_train_128_folder_name)
test_128, test_names = load_images_from_folder(dataset_root + converted_test_128_folder_name)

In [ ]:
# save(save_path + 'train_names.pkl',train_names)
# save(save_path + 'test_names.pkl',test_names)
# save(save_path + 'train_128.pkl',train_128)
# save(save_path + 'test_128.pkl',test_128)

In [ ]:
train_names = load(save_path + 'train_names.pkl')
test_names = load(save_path + 'test_names.pkl')

# Useful functions 

## Display images

In [ ]:
def display_image_converted(img_path, isTrain=True):
  full_path = cv2.imread(os.path.join(dataset_root+ converted_train_128_folder_name,img_path))
  if full_path is None:
    full_path = cv2.imread(os.path.join(dataset_root+ converted_test_128_folder_name,img_path))
  plt.imshow(full_path,cmap='gray')
  plt.xlabel(img_path)
  plt.show()

def display_image_original(img_path, isTrain=True):
  full_path = cv2.imread(os.path.join(dataset_root+ original_train_folder_name,img_path))
  if full_path is None:
    full_path = cv2.imread(os.path.join(dataset_root+ original_test_folder_name,img_path))
  plt.imshow(full_path,cmap='gray')
  plt.xlabel(img_path)
  plt.show()

print(f'No. of images ={len(train_names)}')
# for img_name in train_names[:5]:
#   display_image_original(img_name)
#   display_image_converted(img_name)

No. of images =5288


## SIFT

In [ ]:
def sift_features(images, names):
  descriptor_list = []
  sift_per_image = []
  image_name = []
  for idx, img in enumerate(images):
    kp, des = sift.detectAndCompute(img,None)
    if des is not None:
      descriptor_list.extend(des)
      image_name.extend(names[idx])
      sift_per_image.append(des)
    else:
      print(f'sift not created for {names[idx]}')
  return descriptor_list, sift_per_image, image_name

train_bovw_feature, train_sift, train_sift_names = sift_features(train_128, train_names) 


sift not created for 10a_hello_kitty_050.jpg


In [ ]:

save(save_path + 'train_bovw_feature.pkl',train_bovw_feature)
save(save_path + 'train_sift.pkl',train_sift)
save(save_path + 'train_sift_names.pkl',train_sift_names)

train_bovw_feature = load(save_path + 'train_bovw_feature.pkl')
train_sift = load(save_path + 'train_sift.pkl')
train_sift_names = load(save_path + 'train_sift_names.pkl')

# original list contains extension .jpg
filter_train_img_names = [i[:i.index('.')] for i in train_names]
filter_test_img_names = [i[:i.index('.')] for i in test_names]

## Apply K-means 

In [ ]:
# def kmeans(k, descriptor_list):
#     kmeans = KMeans(n_clusters = k, n_init=10)
#     kmeans.fit(descriptor_list)
#     visual_words = kmeans.cluster_centers_ 
#     return visual_words

# visual_words = kmeans(1000, train_bovw_feature)

In [ ]:
# save(dataset_root + f"visual_words_{x_dim}.pkl", visual_words)
visual_words = load(save_path + 'visual_words.pkl')

## Generating Histograms 

In [ ]:
def input_vector_encoder(feature, codebook):
    code, _ = vq.vq(feature, codebook.tolist())
    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1), normed=True)
    return word_hist

def image_class(all_bovw, centers):
     return [input_vector_encoder(img, centers) for img in all_bovw if img is not None]      

def get_filter_sift(sifts, names):
  filter_sifts, filter_names = [], []
  for sif, name in zip(sifts, names):
    if sif is not None:
      filter_sifts.append(sif)
      filter_names.append(name)
  return filter_sifts, filter_names


train_sift, train_names = get_filter_sift(train_sift, train_names)
hist_train = image_class(train_sift, visual_words) 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
save(save_path + 'hist_train.pkl',hist_train)
hist_train = load(save_path + 'hist_train.pkl')

save(save_path + 'reduce_train_names.pkl',train_names)
save(save_path + 'reduce_train_sift.pkl',train_sift)

reduce_train_names = load(save_path + 'reduce_train_names.pkl')
reduce_train_sift = load(save_path + 'reduce_train_sift.pkl')